# NaCo - Practicum 4
**Benchmarking with IOHexperimenter**

## Focus Today: Using IOH to benchmark your optimisers on custom problems

In this practicum, we will:

1. Learn how to wrap objective functions so they can be used by IOHexperimenter.
2. Adapt one of our own optimisers (like Random Search or Simulated Annealing) to work within the IOH framework.
3. Benchmark and compare the performance of our optimisers on custom and built-in problems using IOH’s logging tools.


**Packages needed today:**

- `ioh` (IOHexperimenter)
- `numpy`
- `math`

If you don’t have IOH yet, install it first:

In [1]:
%pip install ioh


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ioh
import numpy as np
import math

# Task 1: Explore built-in IOH problems

IOH provides a set of well-known benchmark functions such as Sphere, Rastrigin, and Rosenbrock.

In this first exercise, you will:

- Load the Sphere problem using `ioh.get_problem()`
- Sample a random candidate `x`
- Evaluate the function at that point

Hint: You can access the problem’s bounds and dimensionality via:
- `problem.bounds.lb` and `problem.bounds.ub`
- `problem.meta_data.n_variables`

Fill in the missing parts below.

In [3]:
# generate an instance of the Sphere problem with dimension 10
sphere_function = ioh.get_problem('Sphere', dimension=10)

# get bounds and dimension
lb = sphere_function.bounds.lb[0]
ub = sphere_function.bounds.ub[0]
n = sphere_function.meta_data.n_variables

# generate a random candidate and evaluate it
x = np.random.uniform(lb, ub, n)
print(sphere_function(x))

198.18700117284737


# Task 2: Wrap Your Own Objective Functions

IOH also allows you to benchmark custom functions like Rastrigin or the Six-Hump Camel function that we used earlier in the course.

In this task:
1. Define the two functions below.
2. Wrap them into IOH problems using `ioh.wrap_problem()`.

This wrapper needs:
- the function itself
- a name
- lower and upper bounds
- the number of variables

Fill in the blanks to make both custom problems work.

In [4]:
def rastrigin(x):
    n = len(x)
    s = 0
    for i in range(n):
        s += x[i]**2 - 10 * np.cos(2 * math.pi * x[i])
    return 10 * n + s

def six_hump_camel(x):
    x1, x2 = x
    return (4 - 2.1 * x1**2 + (x1**4)/3) * x1**2 + x1*x2 + (-4 + 4*x2**2) * x2**2

def create_custom_ioh_problem(objective_function, name, lb, ub, n, instance=1):
    return ioh.wrap_problem(
        function=objective_function,
        name=name,
        problem_class=ioh.ProblemClass.REAL,
        dimension=2,
        instance=instance,
        optimization_type=ioh.OptimizationType.MIN,
        lb=lb,
        ub=ub,
    )

# Create two custom problems
rastrigin_problem = create_custom_ioh_problem(rastrigin, "Rastrigin", -5.5, 5.5, 2)
six_hump_camel_problem = create_custom_ioh_problem(six_hump_camel, "SixHumpCamel", -1.5, 1.5, 2)

# Task 3: Implement Your Optimisers

IOH expects callable objects (functions or classes implementing `__call__`) that take a problem as input and perform the optimisation.

We will use two algorithms that we already implemented:
- Random Search
- Simulated Annealing

Both optimisers should evaluate the problem using `problem(x)` and track the best found solution.

Complete the missing parts of each class below.

In [5]:
class RandomSearch:
    def __init__(self, budget=1000):
        self.budget = budget

    def __call__(self, problem):
        lb, ub = problem.bounds.lb, problem.bounds.ub
        n_vars = problem.meta_data.n_variables

        best_x = np.random.uniform(lb, ub, size=n_vars)
        best_f = problem(best_x)

        for _ in range(1, self.budget):
            x = np.random.uniform(lb, ub, size=n_vars)
            f = problem(x)
            if f < best_f:
                best_x, best_f = x, f
        return best_x, best_f

In [6]:
class SimulatedAnnealing:
    def __init__(self, T, delta_T, sigma, B):
        self.T = T
        self.delta_T = delta_T
        self.sigma = sigma
        self.B = B
        
    def __call__(self, obj_func):
        n_variables = obj_func.meta_data.n_variables
        lb = obj_func.bounds.lb[0]
        ub = obj_func.bounds.ub[0]
        T = self.T

        x_best = np.random.uniform(lb, ub, size=n_variables)
        f_best = obj_func(x_best)

        for i in range(self.B):
            z = np.random.normal(0, self.sigma, n_variables)
            x = np.clip(x_best + z, lb, ub)
            f = obj_func(x)

            if f < f_best:
                f_best = f
                x_best = x
            elif np.random.uniform(0, 1) < np.exp(-(f - f_best) / T):
                f_best = f
                x_best = x

            T -= self.delta_T
        return x_best, f_best


# Task 4: Attach a Logger and Run a Single Experiment

Before running full benchmarks, let’s see how IOH logging works.

In this task, you will:
- Attach a logger to one of your problems (e.g., Rastrigin)
- Run one optimizer once
- Observe the `IOH_Data_demo` directory being created and files written there


In [7]:
# Create a logger that saves results
logger = ioh.logger.Analyzer(
    root="IOH_Data_demo",
    folder_name="RandomSearch_Rastrigin_demo",
    algorithm_name="RandomSearch",
    store_positions=True,
)

# Attach the logger to the problem
rastrigin_problem.attach_logger(logger)

# Run a single optimizer on the problem
rs = RandomSearch() # instantiate a random search optimiser
best_x, best_f = rs(rastrigin_problem) # run it on the rastrigin problem
print("Best value found:", best_f) # print the best fitness achieved

# Detach the logger and close it
rastrigin_problem.detach_logger()
logger.close()

Best value found: 2.0355452096782436


# Task 5: Log and Analyze Your Experiments

IOH can automatically log every evaluation and save the results to disk.

We’ll use the `Analyzer` logger to record results for each optimizer/problem combination.

Then you can compress the `IOH_Data` folder into a zip file and upload it to the online analyzer: https://iohanalyzer.liacs.nl

Fill in the code below to benchmark both algorithms.


In [8]:
def run_experiments(algorithms, problems, n_repeats=5, root="IOH_Data"):
    for problem_name, ioh_problem in problems.items():
        print(f"=== Running experiments on {problem_name} ===")
        for alg_name, alg in algorithms.items():
            print(f"Starting optimizer: {alg_name}")

            # Create a logger for each algorithm/problem pair
            logger = ioh.logger.Analyzer(
                root=root,
                folder_name=f"{alg_name}_{problem_name}",
                algorithm_name=alg_name,
                algorithm_info=str(alg.__dict__),
                store_positions=True,
            )

            # Attach the logger to the problem
            ioh_problem.attach_logger(logger)

            # Repeat each experiment several times
            for rep in range(1, n_repeats + 1):
                print(f"  Run {rep}/{n_repeats}")
                alg(ioh_problem)
                ioh_problem.reset() # reset the problem for the next run

            # Detach the logger and close it
            ioh_problem.detach_logger()
            logger.close()

# Define algorithms and problems
algorithms = {
    "RandomSearch": RandomSearch(budget=300),
    "SimulatedAnnealing": SimulatedAnnealing(T=0.2, delta_T=0.0001, sigma=0.02, B=300)
}

problems = {
    "Rastrigin": rastrigin_problem,
    "SixHumpCamel": six_hump_camel_problem
}

# Run your experiments (this may take a few seconds)
run_experiments(algorithms, problems, n_repeats=5, root="IOH_Data")

=== Running experiments on Rastrigin ===
Starting optimizer: RandomSearch
  Run 1/5
  Run 2/5
  Run 3/5
  Run 4/5
  Run 5/5
Starting optimizer: SimulatedAnnealing
  Run 1/5
  Run 2/5
  Run 3/5
  Run 4/5
  Run 5/5
=== Running experiments on SixHumpCamel ===
Starting optimizer: RandomSearch
  Run 1/5
  Run 2/5
  Run 3/5
  Run 4/5
  Run 5/5
Starting optimizer: SimulatedAnnealing
  Run 1/5
  Run 2/5
  Run 3/5
  Run 4/5
  Run 5/5


A folder named `IOH_Data` has been created in your working directory. It contains logs for each optimizer/problem pair.

1. Compress this directory (e.g. right-click → “Compress” or use `!zip -r IOH_Data.zip IOH_Data` in a notebook cell).

2. Upload the `.zip` file to https://iohanalyzer.liacs.nl

3. Explore the visualizations:

In [9]:
!zip -r IOH_Data.zip IOH_Data

  adding: IOH_Data/ (stored 0%)
  adding: IOH_Data/RandomSearch_Rastrigin/ (stored 0%)
  adding: IOH_Data/RandomSearch_Rastrigin/data_f3_Rastrigin/ (stored 0%)
  adding: IOH_Data/RandomSearch_Rastrigin/data_f3_Rastrigin/IOHprofiler_f3_DIM2.dat (deflated 50%)
  adding: IOH_Data/RandomSearch_Rastrigin/IOHprofiler_f3_Rastrigin.json (deflated 49%)
  adding: IOH_Data/RandomSearch_Rastrigin-2/ (stored 0%)
  adding: IOH_Data/RandomSearch_Rastrigin-2/data_f3_Rastrigin/ (stored 0%)
  adding: IOH_Data/RandomSearch_Rastrigin-2/data_f3_Rastrigin/IOHprofiler_f3_DIM2.dat (deflated 50%)
  adding: IOH_Data/RandomSearch_Rastrigin-2/IOHprofiler_f3_Rastrigin.json (deflated 49%)
  adding: IOH_Data/SimulatedAnnealing_Rastrigin/ (stored 0%)
  adding: IOH_Data/SimulatedAnnealing_Rastrigin/data_f3_Rastrigin/ (stored 0%)
  adding: IOH_Data/SimulatedAnnealing_Rastrigin/data_f3_Rastrigin/IOHprofiler_f3_DIM2.dat (deflated 52%)
  adding: IOH_Data/SimulatedAnnealing_Rastrigin/IOHprofiler_f3_Rastrigin.json (deflated

# Task 6: Using IOH with Your Own Gym Problems

For your assignment, you will integrate your GymProblem environment with IOH.

IOH expects an objective function that returns a single value.

Our `GymProblem.__call__()` currently returns two values (returns and rewards).  
You must adapt it in one of the following ways:

---
### Option 1: Modify the problem class
In `.play_episode()`, make sure only the returns are returned:
```python
return returns
```
instead of:
```python
return returns, rewards
```

---
### Option 2: Create a wrapper function
```python
def problem_wrapper(gym_problem):
    def wrapped_problem(x):
        returns, _ = gym_problem(x)
        return -returns
    return wrapped_problem

gym_problem = GymProblem()
wrapped_problem = problem_wrapper(gym_problem)
ioh_problem = ioh.wrap_problem(
    function=wrapped_problem,
    name="BaseGymProblem",
    problem_class=ioh.ProblemClass.REAL,
    dimension=gym_problem.n_variables,
    optimization_type=ioh.OptimizationType.MAX,
    lb=-1.0,
    ub=1.0,
)
```

---
Your goal:
- Wrap your GymProblem so it works with IOH.  
- Benchmark it with at least two optimisers.  
- Compare hyperparameters, number of evaluations, and performance across different environment instances.

Don't forget to try different gravities, winds, and turbulence levels!

In [10]:
import gymnasium as gym

class GymProblem:
    def __init__(self, env_name: str = "LunarLander-v3", continuous: bool = False, gravity=-10.0, enable_wind: bool = False, wind_power: float = 0, turbulence_power: float = 0):
        assert env_name in gym.registry
        
        self.simulation_params = {
            "continuous": continuous,
            "gravity": gravity,
            "enable_wind": enable_wind,
            "wind_power": wind_power,
            "turbulence_power": turbulence_power
        }

        self.env_spec = gym.registry[env_name]
        self.env = self.env_spec.make(**self.simulation_params)
        self.state_size = self.env.observation_space.shape[0]
        
        self.continuous = continuous

        if not continuous:
            self.activation = np.argmax
            self.n_outputs = self.env.action_space.n
        else:
            self.activation = np.tanh
            self.n_outputs = self.env.action_space.shape[0]
            
        self.n_variables = self.state_size * self.n_outputs
        self.M = np.zeros((self.state_size, self.n_outputs))

    def play_episode(self, x: np.ndarray, **env_kwargs) -> float:
        self.M = x.reshape(self.state_size, self.n_outputs)
        
        env = self.env_spec.make(**env_kwargs)
        observation, *_ = env.reset()
        
        returns = 0
        rewards = []
        for _ in range(self.env_spec.max_episode_steps):
            action = self.activation(self.M.T @ observation)
            observation, reward, terminated, truncated, info = env.step(action)
            returns += reward
            rewards.append(reward)
            if terminated or truncated:
                break
        
        env.close()
        print(returns)
        return returns # if you want to simply wrap the gym problem with ioh.wrap_problem
        # return returns, rewards

    def __call__(self, x: np.ndarray):
        return self.play_episode(x, **self.simulation_params)

    def show(self, x: np.ndarray) -> float:
        return self.play_episode(x, render_mode="human", **self.simulation_params)

    def sample(self) -> np.ndarray:
        return np.random.uniform(-1, 1, self.n_variables)

ModuleNotFoundError: No module named 'gymnasium'